In [29]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [32]:
from pathlib import Path

path = Path("/Users/nicolasfavrot/Personal/rag-based-solar-inverter/notebooks/data/bataille_de_france/raw/s2-ep1.txt")
raw = path.read_text(encoding="utf-8", errors="ignore")
raw

"# tactiq.io free youtube transcript\n# No title found\n# https://www.youtube.com/watch/1GKzqxKwBK0\n\n00:00:00.200 nous sommes le 27 mars 1796 à nice pendant la révolution française dans les rangs de l'armée républicaine\n00:00:06.859 le bruit court qu'un nouveau général est sur le point de prendre ses quartiers il se dit que ce général et jeunes qu'il\n00:00:13.099 est inexpérimenté mais il se dit aussi qu'il est plein de rêves et plein d'ambition mais pourra-t-il seulement\n00:00:18.260 redonné courage aux troupes pourra-t-il redonner confiance et fierté à ces soldats qui au premier jour du\n00:00:23.450 printemps 1796 sont démoralisés sous-équipés mal payés et en proie à la\n00:00:29.030 mutinerie comment pourra-t-il lui du haut de ses 26 ans mené une campagne militaire dans\n00:00:34.730 ces conditions et voilà que soudain le jeune homme prend la parole et s'adresse à l'armée soldats vous êtes nuls vous\n00:00:41.630 êtes mal nourris le gouvernement vous dois beaucoup il ne peut r

In [30]:
from huggingface_hub import login
login(new_session=False)

In [33]:
from retriever_for_video_transcripts.cleaning import load_txt_folder_as_documents


folder_path = "/Users/nicolasfavrot/Personal/rag-based-solar-inverter/notebooks/data/bataille_de_france/raw/"
docs_before_split = load_txt_folder_as_documents(folder_path)

In [4]:
docs_before_split[2].page_content

"nous sommes le 27 mars 1796 à nice pendant la révolution française dans les rangs de l'armée républicaine le bruit court qu'un nouveau général est sur le point de prendre ses quartiers il se dit que ce général et jeunes qu'il est inexpérimenté mais il se dit aussi qu'il est plein de rêves et plein d'ambition mais pourra-t-il seulement redonné courage aux troupes pourra-t-il redonner confiance et fierté à ces soldats qui au premier jour du printemps 1796 sont démoralisés sous-équipés mal payés et en proie à la mutinerie comment pourra-t-il lui du haut de ses 26 ans mené une campagne militaire dans ces conditions et voilà que soudain le jeune homme prend la parole et s'adresse à l'armée soldats vous êtes nuls vous êtes mal nourris le gouvernement vous dois beaucoup il ne peut rien vous donner mes mois napoléon bonaparte je veux vous conduire dans les plaines les plus fertiles du monde et ses plaines ses plaines si fertile ce sont celles de l'italie un pays où les peuples et les villes n

In [5]:
docs_before_split[0].metadata

{'source': 'rome-ep1.txt',
 'path': '/Users/nicolasfavrot/Personal/rag-based-solar-inverter/notebooks/data/bataille_de_france/raw/rome-ep1.txt',
 'url': 'https://www.youtube.com/watch/mux2wH5fBF0'}

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap  = 50,
)
docs_after_split = text_splitter.split_documents(docs_before_split)

docs_after_split[100]

Document(metadata={'source': 's2-ep1.txt', 'path': '/Users/nicolasfavrot/Personal/rag-based-solar-inverter/notebooks/data/bataille_de_france/raw/s2-ep1.txt', 'url': 'https://www.youtube.com/watch/1GKzqxKwBK0'}, page_content="gras victoire était loin d'être acquise arrivé à l'automne à 1796 en effet l'autriche refuse toujours la défaite elle refuse d'abandonner l'italie alors elle contre attaque encore une fois un certain général alvin six marches avec quarante mille hommes pour libérer mantoue napoléon qui ne peut compter alors que sur vingt-cinq à trente mille soldats ordonne à serrurier de continuer le siège tandis que lui se porte à la rencontre des autrichiens mais voilà que les mauvaises nouvelles s'accumulent son avant-garde s'est fait bousculer et s'est réfugié près de vérone quant au siège de mantoue il est au plus mal car on raconte que les assiégés sont sur le point de briser l'encerclement fall apart")

In [7]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs])//len(docs)
avg_char_before_split = avg_doc_length(docs_before_split)
avg_char_after_split = avg_doc_length(docs_after_split)

print(f'Before split, there were {len(docs_before_split)} documents loaded, with average characters equal to {avg_char_before_split}.')
print(f'After split, there were {len(docs_after_split)} documents (chunks), with average characters equal to {avg_char_after_split} (average chunk length).')

Before split, there were 22 documents loaded, with average characters equal to 27028.
After split, there were 924 documents (chunks), with average characters equal to 688 (average chunk length).


In [8]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="intfloat/multilingual-e5-large",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

/var/folders/x_/sdjp80td4cs0wflptkyqhcs40000gn/T/ipykernel_5666/928891410.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings = HuggingFaceBgeEmbeddings(


In [9]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.02045493  0.00424797 -0.03061799 ...  0.00255879 -0.00493124
 -0.01357661]
Size of the embedding:  (1024,)


In [10]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

In [11]:
query = """Combien de soldats étaient présent lors de la bataille des pyramides ?"""
         
relevant_documents = vectorstore.similarity_search(query, k=1) # Retrieve only 1 document
print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
print(relevant_documents[0].page_content)

There are 1 documents retrieved which are relevant to the query. Display the first one:

du pays réunis toutes ses forces disponibles encore sous le choc d'une invasion qui s'est faite sans prévenir il attend bonaparte de pied ferme à quelques kilomètres seulement des pyramides [Musique] nous sommes le 21 juillet 1798 à proximité du caire en vue des pyramides d'or ce jour là sous la chaleur insupportable de l'état égyptien napoléon bonaparte et ses vingt mille hommes prennent connaissance du terrain et du dispositif adverse leur posant moore ap les attend avec une force comprise entre 20 et 35 milles hommes et qui sont répartis de la façon suivante sur la gauche une masse énorme de cavaliers mamelouks environ dix mille dix mille sommes bien entraînés bien équipé et prêts à


In [12]:
relevant_documents[0].metadata

{'source': 's2-ep2.txt',
 'path': '/Users/nicolasfavrot/Personal/rag-based-solar-inverter/notebooks/data/bataille_de_france/raw/s2-ep2.txt',
 'url': 'https://www.youtube.com/watch/om1yW1eMFjM'}

# From chunk to timestamp

In [ ]:
%load_ext autoreload
%autoreload 2

from retriever_for_video_transcripts.retrieve_timestamp import get_timestamp_for_chunk


retrieved = relevant_documents[0].page_content
retrieved = retrieved[:10] + "aaaa" + retrieved[14:]
source = relevant_documents[0].metadata["source"]

with open(folder_path+source, "r", encoding="utf-8") as f:
    timed_doc = f.read()

ts = get_timestamp_for_chunk(retrieved, timed_doc)
print(ts)


00:12:51.030


In [ ]:
from retriever_for_video_transcripts.urls import make_timed_url


url = relevant_documents[0].metadata["url"]
if ts is not None:
    print(make_timed_url(url, ts))


https://www.youtube.com/watch/om1yW1eMFjM?t=771


# All together

In [ ]:
query = """Qu'est ce que la strategie de la colonne d'attaque ?"""

index_to_look = 0
relevant_documents = vectorstore.similarity_search(query, k=1)
retrieved = relevant_documents[index_to_look].page_content
source = relevant_documents[index_to_look].metadata["source"]

with open(folder_path+source, "r", encoding="utf-8") as f:
    timed_doc = f.read()

ts = get_timestamp_for_chunk(retrieved, timed_doc)

url = relevant_documents[index_to_look].metadata["url"]
if ts is not None:
    print(make_timed_url(url, ts))


https://www.youtube.com/watch/om1yW1eMFjM?t=949


In [16]:
relevant_documents[index_to_look].metadata

{'source': 's2-ep2.txt',
 'path': '/Users/nicolasfavrot/Personal/rag-based-solar-inverter/notebooks/data/bataille_de_france/raw/s2-ep2.txt',
 'url': 'https://www.youtube.com/watch/om1yW1eMFjM'}

In [17]:
retrieved

"un minime a désorganisé les rangs adverses mais quelle est cette stratégie est ce une feinte et soeurs piège où sont ils tous suicidaire bonaparte ne le sait pas il n'a pas le temps de deviner alors il fait armées tous ces canaux disponibles il ordonne que l'on tire sur ses seins prudent cavaliers les artilleurs technique il charge des poulets si rod enquêteurs vont pleuvoir une pluie de plomb et d'obus sur l'adversaire les pauvres mamelons se font dépasser littéralement par le français mais les braves continuent leur route qu'ils suivent les ordres de leur général il se rapproche toujours plus près de la division de seine lui non plus n'en croit pas ses yeux alors ils forment le carré"

In [18]:
query = """Qu'est ce que la colonne d'attaque ?"""
relevant_documents = vectorstore.similarity_search(query, k=3)

In [19]:
relevant_documents[1].page_content

'pas à vous abonner à notre chaîne à laisser un commentaire en bas de la vidéo et rendez vous au prochain épisode'

# Create training questions

In [20]:
s1="s2-ep1.txt"
q1="Comment s'appelle le général qui devait résister aux assauts autrichiens durant la bataille de Rivoli ?"
r1="00:31:50.690"

s2="s2-ep1.txt"
q2="Quand a lieu la prise de Mantoue par Bonaparte ?"
r2="00:35:34.330"

s3="s2-ep1.txt"
q3="Combien de soldats sont détachés initialement pour la campagne d'Italie ?"
r3="00:28:45.760"

#ep2
s4="s2-ep2.txt"
q4="Quels étaient les effectifs français lors de la bataille des pyramides ?"
r4="00:13:20.870"

s5="s2-ep2.txt"
q5="Où est-ce que Lucien Bonaparte veut transférer la chambre des 500 lors du coup d'État de Napoléon ?"
r5="00:31:06.330"

#ep3
s6="s2-ep3.txt"
q6="Comment était divisé le paysage politique en France dans les années 1800 ?"
r6="00:03:56.010"

s7="s2-ep3.txt"
q7="Comment s'est déroulée la bataille de Marengo ?"
r7="00:15:13.900"

#ep4
s8="s2-ep4.txt"
q8="Quand Napoléon a t'il été couronné empereur ?"
r8="00:00:01.890"

s9="s2-ep4.txt"
q9="Quel est le nom du maréchal français qui se battit à Günzburg ?"
r9="00:09:44.279"

#ep5
s10="s2-ep5.txt"
q10="Quelle était la dette publique entre 1802 et 1814 ?"
r10="00:03:56.710"


vars_list = [
    (s1, q1, r1),
    (s2, q2, r2),
    (s3, q3, r3),
    (s4, q4, r4),
    (s5, q5, r5),
    (s6, q6, r6),
    (s7, q7, r7),
    (s8, q8, r8),
    (s9, q9, r9),
    (s10, q10, r10),
]

eval_dataset = []
for s, q, r in vars_list:
    eval_dataset.append({"source": s, "query": q, "answer": r})

eval_dataset[0] 

{'source': 's2-ep1.txt',
 'query': "Comment s'appelle le général qui devait résister aux assauts autrichiens durant la bataille de Rivoli ?",
 'answer': '00:31:50.690'}

In [ ]:
from retriever_for_video_transcripts.evaluate import evaluate_query


index_query = 1
evaluate_query(
    eval_dataset[index_query]["query"],
    eval_dataset[index_query]["source"],
    eval_dataset[index_query]["answer"],
    vectorstore,
    folder_path
)

{'question': 'Quand a lieu la prise de Mantoue par Bonaparte ?',
 'gt_source': 's2-ep1.txt',
 'pred_source': 's2-ep1.txt',
 'source_correct': True,
 'gt_ts': '00:35:34.330',
 'pred_ts': '00:35:03.100',
 'ts_error_sec': 31,
 'pred_url': 'https://www.youtube.com/watch/1GKzqxKwBK0?t=2103'}

In [52]:
for query in eval_dataset:
    answer = evaluate_query(
        query["query"],
        query["source"],
        query["answer"],
        vectorstore,
        folder_path
    )
    print(answer["question"],answer["pred_url"], answer["source_correct"], answer["ts_error_sec"], )

Comment s'appelle le général qui devait résister aux assauts autrichiens durant la bataille de Rivoli ? https://www.youtube.com/watch/B9yvF3y4mPc?t=629 False 1281
Quand a lieu la prise de Mantoue par Bonaparte ? https://www.youtube.com/watch/1GKzqxKwBK0?t=2103 True 31
Combien de soldats sont détachés initialement pour la campagne d'Italie ? https://www.youtube.com/watch/1GKzqxKwBK0?t=1725 True 0
Quels étaient les effectifs français lors de la bataille des pyramides ? https://www.youtube.com/watch/om1yW1eMFjM?t=771 True 29
Où est-ce que Lucien Bonaparte veut transférer la chambre des 500 lors du coup d'État de Napoléon ? https://www.youtube.com/watch/om1yW1eMFjM?t=2116 True 250
Comment était divisé le paysage politique en France dans les années 1800 ? https://www.youtube.com/watch/B9yvF3y4mPc?t=191 True 45
Comment s'est déroulée la bataille de Marengo ? https://www.youtube.com/watch/B9yvF3y4mPc?t=984 True 71
Quand Napoléon a t'il été couronné empereur ? https://www.youtube.com/watch/O7s

In [44]:
answer

{'question': 'Quelle était la dette publique entre 1802 et 1814 ?',
 'gt_source': 's2-ep5.txt',
 'pred_source': 's2-ep5.txt',
 'source_correct': True,
 'gt_ts': '00:03:56.710',
 'pred_ts': '00:03:34.010',
 'ts_error_sec': 22}

# Data preprocessing with vocab:

In [ ]:
# napoleon_cleaner.py
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Iterable
import json, re, unicodedata
from rapidfuzz import process, fuzz

try:
    import spacy
    _NLP = spacy.load("fr_core_news_md")
except Exception:
    _NLP = None  # runs without spaCy

# ---------- 0) Utils ----------
def strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def normalize(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s

def tokenize(text: str) -> List[str]:
    return re.findall(r"\w+|[^\w\s]", text, re.UNICODE)

def ngrams(tokens: List[str], n: int) -> Iterable[Tuple[int, int, str]]:
    for i in range(len(tokens)):
        for k in range(1, n+1):
            if i+k <= len(tokens):
                yield i, i+k, "".join(tokens[i:i+k]) if k==1 else " ".join(tokens[i:i+k])

# Simple French-ish phonetic key (fallback). Good enough for many ASR slips.
def phonetic_key(word: str) -> str:
    w = strip_accents(word.lower())
    w = re.sub(r"ph", "f", w)
    w = re.sub(r"[aeiouy]+", "a", w)  # collapse vowels
    w = re.sub(r"[ckq]+", "k", w)
    w = re.sub(r"gn", "n", w)
    w = re.sub(r"ch", "sh", w)
    w = re.sub(r"[^a-z]", "", w)
    return w

# ---------- 1) Gazetteer model ----------
@dataclass
class GazetteerItem:
    name: str
    kind: str          # "ville" | "bataille" | "personne" | "terme"
    variants: List[str]
    region: Optional[str] = None  # e.g., "Piémont", "Ligurie", ...

class GazetteerIndex:
    def __init__(self, gz_json: Dict):
        self.items: List[GazetteerItem] = []
        # Villes
        for v in gz_json.get("villes", []):
            self.items.append(GazetteerItem(v["nom"], "ville", v.get("variantes", [])))
        # Batailles
        for b in gz_json.get("batailles", []):
            self.items.append(GazetteerItem(b, "bataille", []))
        # Personnalités
        for p in gz_json.get("personnalites", {}).get("francais", []):
            self.items.append(GazetteerItem(p, "personne", []))
        for p in gz_json.get("personnalites", {}).get("etrangers", []):
            self.items.append(GazetteerItem(p, "personne", []))
        # Termes
        for t in gz_json.get("unites_termes", []):
            self.items.append(GazetteerItem(t, "terme", []))

        # flat dictionaries for lookups
        self.surface_forms: List[str] = []
        self.id_by_surface: Dict[str, int] = {}
        self.kind_by_id: Dict[int, str] = {}
        self.norm_by_surface: Dict[str, str] = {}
        self.phon_by_surface: Dict[str, str] = {}

        for idx, it in enumerate(self.items):
            forms = [it.name] + it.variants
            for f in forms:
                sf = f
                self.surface_forms.append(sf)
                self.id_by_surface[sf] = idx
                self.kind_by_id[idx] = it.kind
                self.norm_by_surface[sf] = strip_accents(sf.lower())
                self.phon_by_surface[sf] = phonetic_key(sf)

        # prebuild lists per kind (for faster fuzzy searches)
        self.forms_by_kind: Dict[str, List[str]] = {}
        for sf in self.surface_forms:
            k = self.kind_by_id[self.id_by_surface[sf]]
            self.forms_by_kind.setdefault(k, []).append(sf)

# ---------- 2) Candidate search ----------
@dataclass
class Candidate:
    replacement: str   # surface from gazetteer
    kind: str
    score_edit: float
    score_phon: float
    score_ctx: float
    score_cap: float
    score_total: float

CTX_PATTERNS = [
    # (regex, expected kind, context bonus)
    (re.compile(r"\b(bataille|combat|siège) d[eu]?\s+$", re.I), "bataille", 0.25),
    (re.compile(r"\b(général|gén\.)\s+$", re.I), "personne", 0.25),
    (re.compile(r"\b(à|au|en|de|du|dans)\s+$", re.I), "ville", 0.25),
]

def context_expected_kind(left_context: str) -> Optional[str]:
    for rx, kind, _ in CTX_PATTERNS:
        if rx.search(left_context):
            return kind
    return None

def build_left_context(tokens: List[str], start: int) -> str:
    left = " ".join(tokens[max(0, start-4):start])
    return left + " "

def fuzzy_candidates(seg: str, pool: List[str], topk: int = 8) -> List[Tuple[str, float]]:
    # RapidFuzz: returns [(match, score, _), ...]
    results = process.extract(seg, pool, scorer=fuzz.WRatio, limit=topk)
    return [(m[0], m[1]/100.0) for m in results if m[1] >= 70]  # threshold 0.70

def phonetic_candidates(seg: str, pool: List[str], phon_by_surface: Dict[str, str]) -> List[str]:
    key = phonetic_key(seg)
    return [p for p in pool if phon_by_surface[p] in key]

def score_candidate(seg: str, repl: str, expected_kind: Optional[str], gz: GazetteerIndex, left_ctx: str) -> Candidate:
    kind = gz.kind_by_id[gz.id_by_surface[repl]]
    # Edit score via WRatio (already computed in fuzzy; recompute here quickly)
    edit = fuzz.WRatio(seg, repl) / 100.0
    # Phonetic
    phon = 1.0 if phonetic_key(seg) == gz.phon_by_surface[repl] else 0.0
    # Context
    ctx_bonus = 0.0
    if expected_kind and expected_kind == kind:
        ctx_bonus += 0.25
    else:
        # lightweight context: prepositions favor cities
        if re.search(r"\b(à|au|en|de|du|dans)\b\s*$", left_ctx, re.I) and kind == "ville":
            ctx_bonus += 0.2
    # Casing/diacritics (reward fixing them)
    cap = 1.0 if repl[:1].isupper() else 0.5

    total = 0.45*edit + 0.25*phon + 0.25*ctx_bonus + 0.05*cap
    return Candidate(replacement=repl, kind=kind, score_edit=edit, score_phon=phon, score_ctx=ctx_bonus, score_cap=cap, score_total=total)

def find_suspects(tokens: List[str], text: str) -> List[Tuple[int, int, str]]:
    suspects = []
    # 1) NER if available
    if _NLP is not None:
        doc = _NLP(text)
        for ent in doc.ents:
            if ent.label_ in {"LOC", "PER"}:
                suspects.append((ent.start_char, ent.end_char, ent.text))
    # 2) Heuristics on tokens/n-grams (up to tri-grams)
    for i, j, seg in ngrams(tokens, 2):
        # suspect: lowercase after preposition; unknown blobs; glued tokens
        if re.match(r"^[a-zàâçéèêëîïôûùüÿñœ'-]+(?: [a-zàâçéèêëîïôûùüÿñœ'-]+)?$", seg):
            left = " ".join(tokens[max(0, i-2):i])
            if re.search(r"\b(à|au|en|de|du|dans|général|bataille|siège)\b", left, re.I):
                suspects.append((i, j, seg))
    # Deduplicate by span
    uniq = {}
    for s in suspects:
        uniq[(s[0], s[1], s[2].lower())] = s
    # Convert token spans to text spans when needed
    final = []
    for _, s in uniq.items():
        final.append(s)
    return final

# ---------- 3) Main correction ----------
def correct_text(text: str, gz_json: Dict, topn_per_span: int = 3,
                 accept_threshold: float = 0.85, soft_threshold: float = 0.70) -> Dict:
    gz = GazetteerIndex(gz_json)
    text = normalize(text)
    toks = tokenize(text)

    # For reconstruction, keep a parallel list we can edit
    out_tokens = toks[:]
    decisions = []

    # Work over token n-gram suspects
    suspects = []
    # Build mapping from token index → char offset (to get left-context)
    char_pos = []
    pos = 0
    for t in toks:
        char_pos.append(pos)
        pos += len(t)
        if not re.match(r"\s|[^\w\s]", t):  # approx spacing; real spacing handled by join later
            pos += 1

    for i, j, seg in ngrams(toks, 2):
        left_ctx = " ".join(toks[max(0, i-4):i]) + " "
        # choose expected kind from context
        exp_kind = context_expected_kind(left_ctx)

        # choose candidate pool based on expected kind (or all)
        if exp_kind and exp_kind in gz.forms_by_kind:
            pool = gz.forms_by_kind[exp_kind]
        else:
            pool = gz.surface_forms

        fz = fuzzy_candidates(seg, pool, topk=8)
        ph = phonetic_candidates(seg, pool, gz.phon_by_surface)

        pool_names = set([m for m,_ in fz] + ph)

        if not pool_names:
            continue

        scored = [score_candidate(seg, name, exp_kind, gz, left_ctx) for name in pool_names]
        scored.sort(key=lambda c: c.score_total, reverse=True)
        topk = scored[:topn_per_span]

        # Decide replacement for that span if confident and if looks like a name
        best = topk[0]
        if best.score_total >= soft_threshold and re.search(r"[A-Za-zÀ-ÿ]", seg):
            # Replace tokens i..j with candidate (single token if space inside, keep as multiple tokens)
            repl_tokens = best.replacement.split(" ")
            out_tokens[i:j] = repl_tokens
            decisions.append({
                "span_tokens": (i, j),
                "original": seg,
                "replacement": best.replacement,
                "kind": best.kind,
                "score": round(best.score_total, 3),
                "status": "auto" if best.score_total >= accept_threshold else "low_conf"
            })

    # Post: fix “à/au/en + Ville” capitalization if missed
    for k in range(len(out_tokens)-1):
        if re.match(r"^(à|au|en|de|du|dans)$", out_tokens[k], re.I) and re.match(r"^[a-zà-ÿ-]+$", out_tokens[k+1]):
            # Try exact match in gazetteer with capitalized first letter
            cap = out_tokens[k+1][:1].upper() + out_tokens[k+1][1:]
            if cap in gz.id_by_surface:
                out_tokens[k+1] = cap

    # Rebuild string (simple join, then whitespace tidy)
    out = " ".join(out_tokens)
    out = re.sub(r"\s+([,.!?;:])", r"\1", out)
    out = re.sub(r"\s+'", "'", out)
    out = re.sub(r"\s+", " ", out).strip()

    return {"text_out": out, "decisions": decisions}


In [ ]:
import json

GZ_PATH = "/Users/nicolasfavrot/Personal/rag-based-solar-inverter/notebooks/data/bataille_de_france/gazetteer/ep1.json"

with open(GZ_PATH, "r", encoding="utf-8") as f:
    GZ = json.load(f)


# 2) Run a correction
inp = ("la situation sur le front italien n'est pas vraiment favorable à bonaparte "
       "sur la droite du dispositif coalisés on trouve les sardes avec 30000 hommes "
       "répartis à mondovi eacce bas")

res = correct_text(inp, GZ)
print(res["text_out"])
for d in res["decisions"]:
    print(d)